In [3]:
# Indicadores que queremos encontrar
# VPA - Valor Patrimonial por Ação - 1.5x
# LPA - Lucro por Ação - 15x
# (22.5 * VPA * LPA) ** (1/2)

In [4]:
# mocking - requests em massa
# zipfiles - biblioteca arquivo zip
# scrapy - biblioteca de scrapping
# https://dados.cvm.gov.br/dataset/cia_aberta-doc-itr - site

In [8]:
import fundamentus

tickers = fundamentus.get_resultado()
list_tickers = list(tickers.index)

c:\Users\Andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\fundamentus\resultado.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]


Multiples,cotacao,pl,pvp,psr,dy,pa,pcg,pebit,pacl,evebit,evebitda,mrgebit,mrgliq,roic,roe,liqc,liq2m,patrliq,divbpatr,c5y
papel,,,,,,,,,,,,,,,,,,,,
AALR3,10.66,-4.79,1.24,1.093,0.0000,0.459,-5.64,59.10,-1.16,94.57,14.59,0.0185,-0.2210,0.0091,-0.2592,0.74,1007410.0,1.015110e+09,1.04,0.0354
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.1429,0.00,0.0,5.721810e+09,0.00,0.7094
ABCB4,21.68,6.00,0.86,0.000,0.0556,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.1429,0.00,12104200.0,5.721810e+09,0.00,0.7094
ABEV3,13.39,13.99,2.34,2.559,0.0569,1.530,56.91,12.41,-23.97,11.59,8.53,0.2063,0.1882,0.1650,0.1671,1.11,335267000.0,9.025490e+10,0.04,0.1323
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,0.0278,-0.0082,2.09,0.0,2.920600e+08,1.31,0.1641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM4,30.88,10.64,1.68,0.531,0.0383,1.362,3.11,8.81,3.65,8.09,7.82,0.0603,0.0499,0.1834,0.1577,4.54,84750.9,6.698980e+08,0.01,0.2265
WMBY3,25.39,-19.30,2.87,0.836,0.0000,0.182,1.20,8.62,-1.50,25.02,25.02,0.0970,-0.0705,0.0239,-0.1486,1.44,0.0,2.124390e+08,6.51,-0.1448
WSON33,67.00,8.07,0.98,1.067,0.0000,0.400,13.41,2.42,-0.89,5.08,5.08,0.4414,0.1358,0.1836,0.1217,1.26,0.0,2.148530e+09,1.21,0.0523


In [13]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

def extrair_indicadores(cell):
    indicador_data = {}
    
    indicador_data['nome'] = cell.find('span', class_='d-flex').text.strip("\n")
    
    valor_span = cell.find('div', class_='value').find('span')
    if valor_span:
        indicador_data['valor'] = valor_span.text.strip()
    
    return indicador_data

def get_data(ticker):
    url_info = f"https://investidor10.com.br/acoes/{ticker.lower()}/"
    
    headers = {"User-Agent": UserAgent().random}

    response = requests.get(url_info, headers=headers)
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.content, "html.parser")
    table_indicators = soup.find("div", id="table-indicators")

    data = {}

    for cell in table_indicators.find_all("div", class_="cell"):
        indicador_data = extrair_indicadores(cell)
        data[indicador_data['nome']] = indicador_data['valor']

    return data


In [15]:
info = {}
for ticker in list_tickers[:100]:
    data = get_data(ticker)
    if data:
        try:
            vpa = float(data["VPA "].replace(".", "").replace(",", "."))
            lpa = float(data["LPA "].replace(".", "").replace(",", "."))
            result = (22.5 * vpa * lpa) ** 0.5
        except ValueError:
            continue
        try:
            conta = round(conta, 2)
        except TypeError:
            pass
        if conta > tickers.loc[ticker]["cotacao"]:
            info[ticker] = {"resultado": conta, "cotacao": tickers.loc[ticker]["cotacao"], "diferenca": round(conta - tickers.loc[ticker]["cotacao"], 2)}
            print(f"{ticker}:\n    Resultado: {round(conta, 2)}\n    Cotação: {tickers.loc[ticker]['cotacao']}\n    Diferença: {round(conta - tickers.loc[ticker]['cotacao'], 2)}\n")

KeyboardInterrupt: 

In [30]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

url = "https://investidor10.com.br/acoes/petr4/"

headers = {"User-Agent": UserAgent().random}

response = req.get(url, headers=headers)
response.raise_for_status()

df = pd.read_html(response.content)
for table in df:
    display(table)'

,Tipo,Data COM,Pagamento,Valor
0,Dividendos,21/11/2023,20/03/2024,67267350
1,Dividendos,21/11/2023,20/02/2024,24328720
2,JSCP,21/11/2023,20/02/2024,42938680
3,Dividendos,27/04/2023,27/12/2023,49806828
4,Dividendos,21/08/2023,15/12/2023,57465200
...,...,...,...,...
87,JSCP,29/06/2001,22/08/2001,102000000
88,JSCP,23/03/2001,04/05/2001,170500000
89,Dividendos,23/03/2001,04/05/2001,60500000
90,Dividendos,23/03/2000,19/05/2000,373500000


,Unnamed: 0,P/L,P/VP,ROE,DY,Valor de Mercado,Margem Líquida
0,PETR4,340,121,"35,47%","20,00%","R$ 476,68 B","25,53%"
1,PRIO3,867,307,"35,33%",-,"R$ 39,54 B","44,93%"
2,CSAN3,-7211,180,"-2,50%","2,41%","R$ 33,32 B","-1,14%"
3,UGPA3,1319,225,"17,03%","1,37%","R$ 28,54 B","1,68%"
4,VBBR3,1383,208,"15,01%","3,39%","R$ 28,13 B","1,24%"
5,ENAT3,5919,106,"1,78%","0,97%","R$ 4,09 B","4,27%"
6,RPMG3,-17,-3,"-16,32%",-,"R$ 142,15 M","-16,72%"


,0,1
0,Nome da Empresa:,PETROLEO BRASILEIRO S.A. PETROBRAS
1,CNPJ:,33.000.167/0001-01
2,Ano de estreia na bolsa:,1977
3,Número de funcionários:,61550
4,Ano de fundação:,1953
